In [0]:
####################################
# CONFIGURATION
####################################

jdbc_url = "jdbc:sqlserver://shameenaserver.database.windows.net:1433;database=BankingDW;encrypt=true;"
jdbc_user = "shameena@shameenaserver"
jdbc_password = "snd@786123"

connection_properties = {
    "user": jdbc_user,
    "password": jdbc_password,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

silver_path = "dbfs:/FileStore/tables/fact_transaction.csv"


In [0]:
from pyspark.sql.functions import col, trim, to_timestamp, when

df = spark.read.option("header", True).option("inferSchema", True).csv(silver_path)

print("Loaded SILVER schema:")
df.printSchema()

# Clean NULL values and trim spaces
string_cols = [f.name for f in df.schema.fields if f.dataType.simpleString() == "string"]

for c in string_cols:
    df = df.withColumn(c, trim(col(c)))


# Identify final columns
amount_column = None
for c in ["Amount", "TransactionAmount", "Amount8", "amount26"]:
    if c in df.columns:
        amount_column = c
        break

df = df.withColumn("Amount_clean", when(col(amount_column) == "null", None).otherwise(col(amount_column)).cast("double"))

# Timestamp standardization
timestamp_col = "processedAt" if "processedAt" in df.columns else "TxnTimestamp"

df = df.withColumn("ProcessedAt", to_timestamp(col(timestamp_col)))

display(df.limit(5))


Loaded SILVER schema:
root
-- Location: string (nullable = true)
-- ATMID: string (nullable = true)
-- processedAt: timestamp (nullable = true)
-- AccountNumber: integer (nullable = true)
-- CustomerID: string (nullable = true)
-- TransactionTime: string (nullable = true)
-- TransactionType: string (nullable = true)
-- TransactionAmount: string (nullable = true)
-- Amount8: integer (nullable = true)
-- id: string (nullable = true)
-- txn_type: string (nullable = true)
-- fraud_flags: string (nullable = true)
-- TransactionID: string (nullable = true)
-- Status: string (nullable = true)
-- txn_time: string (nullable = true)
-- source: string (nullable = true)
-- TxnID: string (nullable = true)
-- TxnTimestamp: string (nullable = true)
-- TxnType18: string (nullable = true)
-- PayeeUPI: string (nullable = true)
-- EventID: string (nullable = true)
-- GeoLocation: string (nullable = true)
-- PayerUPI: string (nullable = true)
-- DeviceID: string (nullable = true)
-- txnType24: string (nullable = true)
-- sourceFile: string (nullable = true)
-- amount26: string (nullable = true)
-- id_1: string (nullable = true)
-- alertType: string (nullable = true)
-- alertTime: string (nullable = true)
-- txn_id: string (nullable = true)
-- alert_time: string (nullable = true)

Location,ATMID,ProcessedAt,AccountNumber,CustomerID,TransactionTime,TransactionType,TransactionAmount,Amount8,id,txn_type,fraud_flags,TransactionID,Status,txn_time,source,TxnID,TxnTimestamp,TxnType18,PayeeUPI,EventID,GeoLocation,PayerUPI,DeviceID,txnType24,sourceFile,amount26,id_1,alertType,alertTime,txn_id,alert_time,Amount_clean
Kolkata,ATM019,2025-12-08T09:47:29.872+0000,1002003396,CUST214,2025-01-01T03:27:00Z,WITHDRAWAL,2000,2000,ATM000208,ATM,[],ATM000208,SUCCESS,null,ATM,null,null,null,null,null,null,null,null,ATM,atm/upi_transaction.csv,45000,ATM000142_Large ATM withdrawal,LARGE ATM WITHDRAWAL,2025-12-08T09:47:18.362209,ATM000142,null,2000.0
Kolkata,ATM020,2025-12-08T09:48:14.848+0000,1002003252,CUST439,2025-01-01T07:10:00Z,WITHDRAWAL,2000,2000,ATM000431,ATM,[],ATM000431,SUCCESS,null,ATM,null,null,null,null,null,null,null,null,ATM,atm/upi_transaction.csv,45000,ATM001419_Large ATM withdrawal,LARGE ATM WITHDRAWAL,2025-12-08T09:51:41.437230,ATM001419,null,2000.0
Hyderabad,ATM027,2025-12-08T09:48:17.468+0000,1002003327,CUST347,2025-01-01T07:24:00Z,WITHDRAWAL,2000,2000,ATM000445,ATM,[],ATM000445,SUCCESS,null,ATM,null,null,null,null,null,null,null,null,ATM,atm/upi_transaction.csv,45000,ATM001434_Large ATM withdrawal,LARGE ATM WITHDRAWAL,2025-12-08T09:51:44.444739,ATM001434,null,2000.0
Pune,ATM008,2025-12-08T09:48:23.283+0000,1002003162,CUST497,2025-01-01T07:52:00Z,WITHDRAWAL,2000,2000,ATM000473,ATM,[],ATM000473,SUCCESS,null,ATM,null,null,null,null,null,null,null,null,ATM,atm/upi_transaction.csv,45000,ATM001084_Large ATM withdrawal,LARGE ATM WITHDRAWAL,2025-12-08T09:50:27.957522,ATM001084,null,2000.0
Pune,ATM024,2025-12-08T09:48:46.271+0000,1002003305,CUST079,2025-01-01T09:31:00Z,WITHDRAWAL,2000,2000,ATM000572,ATM,[],ATM000572,SUCCESS,null,ATM,null,null,null,null,null,null,null,null,ATM,atm/upi_transaction.csv,45000,ATM001084_Large ATM withdrawal,LARGE ATM WITHDRAWAL,2025-12-08T09:50:27.957522,ATM001084,null,2000.0


In [0]:
from pyspark.sql.functions import to_date, dayofmonth, month, year, weekofyear, quarter

df_date = df.withColumn("DateValue", to_date(col("ProcessedAt"))) \
            .withColumn("Day", dayofmonth(col("ProcessedAt"))) \
            .withColumn("Month", month(col("ProcessedAt"))) \
            .withColumn("Year", year(col("ProcessedAt"))) \
            .withColumn("Week", weekofyear(col("ProcessedAt"))) \
            .withColumn("Quarter", quarter(col("ProcessedAt"))) \
            .select("DateValue","Day","Month","Year","Week","Quarter") \
            .dropDuplicates()

df_date.write.format("jdbc").mode("append") \
    .option("url", jdbc_url).option("dbtable", "DimDate") \
    .options(**connection_properties).save()

print("DimDate Loaded ✔️")


DimDate Loaded ✔️

In [0]:
from pyspark.sql.functions import lit
from pyspark.sql.types import DateType, BooleanType

df_customer_stg = df.select("CustomerID", "ProcessedAt") \
    .dropDuplicates() \
    .withColumn("EffectiveDate", to_date(col("ProcessedAt"))) \
    .withColumn("ExpiryDate", lit(None).cast(DateType())) \
    .withColumn("IsActive", lit(True).cast(BooleanType())) \
    .select("CustomerID","EffectiveDate","ExpiryDate","IsActive")

df_customer_stg.write.format("jdbc").mode("append") \
    .option("url", jdbc_url).option("dbtable", "DimCustomer_Staging") \
    .options(**connection_properties).save()

print("DimCustomer_Staging Loaded ✔️")


DimCustomer_Staging Loaded ✔️

In [0]:
df_account_stg = df.select(col("AccountNumber").cast("string"), "CustomerID", "ProcessedAt") \
    .dropDuplicates() \
    .withColumn("EffectiveDate", to_date(col("ProcessedAt"))) \
    .withColumn("ExpiryDate", lit(None).cast(DateType())) \
    .withColumn("IsActive", lit(True).cast(BooleanType())) \
    .select("AccountNumber","CustomerID","EffectiveDate","ExpiryDate","IsActive")

df_account_stg.write.format("jdbc").mode("append") \
    .option("url", jdbc_url).option("dbtable", "DimAccount_Staging") \
    .options(**connection_properties).save()

print("DimAccount_Staging Loaded ✔️")


DimAccount_Staging Loaded ✔️

In [0]:
df_branch = df.select(col("ATMID"), col("Location").alias("BranchName")).dropDuplicates()

df_branch.write.format("jdbc").mode("append") \
    .option("url", jdbc_url).option("dbtable", "DimBranch_Staging") \
    .options(**connection_properties).save()

print("DimBranch_Staging Loaded ✔️")


DimBranch_Staging Loaded ✔️

In [0]:
product_column = None
for c in ["TransactionType", "txn_type", "TxnType"]:
    if c in df.columns:
        product_column = c
        break

df_product = df.select(col(product_column).alias("ProductName")).dropDuplicates()

df_product.write.format("jdbc").mode("append") \
    .option("url", jdbc_url).option("dbtable", "DimProduct_Staging") \
    .options(**connection_properties).save()

print("DimProduct_Staging Loaded ✔️")


DimProduct_Staging Loaded ✔️

In [0]:
df_fact = df.select(
    col("TransactionID"), 
    col("CustomerID"),
    col("AccountNumber").cast("string"),
    col("ATMID"),
    col(product_column).alias("TransactionType"),
    col("Amount_clean").alias("TransactionAmount"),
    col("ProcessedAt").alias("TransactionTimestamp"),
    col("Status"),
    col("fraud_flags")
).dropDuplicates(["TransactionID"])

df_fact.write.format("jdbc").mode("append") \
    .option("url", jdbc_url).option("dbtable", "FactTransactions_Staging") \
    .options(**connection_properties).save()

print("FactTransactions_Staging Loaded ✔️")


FactTransactions_Staging Loaded ✔️

In [0]:
from pyspark.sql.functions import count

# Fraud records where fraud_flags is not empty or null
df_fraud = df_fact.filter((col("fraud_flags").isNotNull()) & (col("fraud_flags") != "[]"))

df_fraud.write.format("jdbc").mode("append") \
    .option("url", jdbc_url).option("dbtable", "FactFraudDetection_Staging") \
    .options(**connection_properties).save()

print("FactFraudDetection_Staging Loaded ✔️")


# Customer Activity Count
df_activity = df_fact.groupBy("CustomerID").agg(count("TransactionID").alias("TransactionCount"))

df_activity.write.format("jdbc").mode("append") \
    .option("url", jdbc_url).option("dbtable", "FactCustomerActivity_Staging") \
    .options(**connection_properties).save()

print("FactCustomerActivity_Staging Loaded ✔️")


FactFraudDetection_Staging Loaded ✔️
FactCustomerActivity_Staging Loaded ✔️